In [1]:
from sqlalchemy import create_engine
from sqlalchemy import inspect
import pandas as pd

engine = create_engine("sqlite:///mta_data.db")
inspect(engine).get_table_names()

df = pd.read_sql('mta_data', engine)

In [2]:
df.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/22/2021,00:00:00,REGULAR,7575503,2588978
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/22/2021,04:00:00,RECOVR AUD,7575515,2588982
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/22/2021,08:00:00,REGULAR,7575523,2588989
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/22/2021,12:00:00,REGULAR,7575557,2588996
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/22/2021,16:00:00,REGULAR,7575630,2589008


In [3]:
df.dtypes

C/A         object
UNIT        object
SCP         object
STATION     object
LINENAME    object
DIVISION    object
DATE        object
TIME        object
DESC        object
ENTRIES      int64
EXITS        int64
dtype: object

In [4]:
df.info

<bound method DataFrame.info of             C/A  UNIT       SCP        STATION LINENAME DIVISION        DATE  \
0          A002  R051  02-00-00          59 ST  NQR456W      BMT  05/22/2021   
1          A002  R051  02-00-00          59 ST  NQR456W      BMT  05/22/2021   
2          A002  R051  02-00-00          59 ST  NQR456W      BMT  05/22/2021   
3          A002  R051  02-00-00          59 ST  NQR456W      BMT  05/22/2021   
4          A002  R051  02-00-00          59 ST  NQR456W      BMT  05/22/2021   
...         ...   ...       ...            ...      ...      ...         ...   
14096273  TRAM2  R469  00-03-01  RIT-ROOSEVELT        R      RIT  02/02/2018   
14096274  TRAM2  R469  00-03-01  RIT-ROOSEVELT        R      RIT  02/02/2018   
14096275  TRAM2  R469  00-03-01  RIT-ROOSEVELT        R      RIT  02/02/2018   
14096276  TRAM2  R469  00-03-01  RIT-ROOSEVELT        R      RIT  02/02/2018   
14096277  TRAM2  R469  00-03-01  RIT-ROOSEVELT        R      RIT  02/02/2018   

       

In [5]:
df["DATE"] = pd.to_datetime(df["DATE"])
df["YEAR"] = df.DATE.dt.year
df["MONTH"] = df.DATE.dt.month

In [12]:
df.head(3)

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,YEAR,MONTH
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,2021-05-22,00:00:00,REGULAR,7575503,2588978,2021,5
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,2021-05-22,04:00:00,RECOVR AUD,7575515,2588982,2021,5
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,2021-05-22,08:00:00,REGULAR,7575523,2588989,2021,5


In [13]:
#df_final_totals = df[df["DATE"].dt.month == 5 & df["DATE"].dt.day == 31]
#df_final_totals

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [6]:
df_sum_yr = df.groupby("YEAR").sum()
df_sum_yr

,ENTRIES,EXITS,MONTH
YEAR,,,
2018,131832087872725,108210482270663,11251243
2019,141770531474115,116540356884039,11525042
2020,160187557739288,131322574473105,12485621
2021,149370031089834,120054901638937,12277136


In [16]:
df_sum_yr_entries = df_sum_yr.iloc[:,0:1]
#df_sum_yr_entries["Δ prev year"] = ["-",df_sum_yr_entries.iloc[1]-df_sum_yr_entries.iloc[0],
                                   # df_sum_yr_entries.iloc[2]-df_sum_yr_entries.iloc[1],
                                    #df_sum_yr_entries.iloc[3]-df_sum_yr_entries.iloc[2]]
df_sum_yr_entries["Δ prev year"] = ["-",141770531474115-131832087872725,160187557739288	-141770531474115,149370031089834-160187557739288]
df_sum_yr_entries

<ipython-input-16-ef30f203087f>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sum_yr_entries["Δ prev year"] = ["-",141770531474115-131832087872725,160187557739288	-141770531474115,149370031089834-160187557739288]


,ENTRIES,Δ prev year
YEAR,,
2018,131832087872725,-
2019,141770531474115,9938443601390
2020,160187557739288,18417026265173
2021,149370031089834,-10817526649454


In [7]:
df_sum_yyyymm = df.groupby(["YEAR","MONTH"]).sum()
df_sum_yyyymm

ENTRIES           EXITS
YEAR MONTH                                
2018 1       5701930258493   4647261548560
     2      31376675229584  25365897876966
     3      34221887650889  28252219826683
     4      32762429420788  27167505699355
     5      27769165312971  22777597319099
2019 1       6990382802923   5749823438663
     2      33079670145293  27332932955093
     3      36483341446430  30013084390016
     4      35962197296279  29453771787890
     5      29254939783190  23990744312377
2020 1       8725378965489   7139833705397
     2      36475645587875  29990401297201
     3      39448326189522  32320530533729
     4      38372961814159  31460688747527
     5      37165245182243  30411120189251
2021 1       2532397296442   2034177063786
     2      35446107277082  28449722614090
     3      38864801074065  31260769059840
     4      37449106662976  30135407337024
     5      35077618779269  28174825564197

In [18]:
columns = [2018,2019,2020,2021]
data = [32762429420788-31376675229584,35962197296279-33079670145293,38372961814159-36475645587875,37449106662976-35446107277082]
print(data)

[1385754191204, 2882527150986, 1897316226284, 2002999385894]
